In [22]:
import numpy as np 
from sklearn.metrics import pairwise_distances

In [38]:
    def build_gaussian_kernel(traj,derivative=False):
        original_shape=np.shape(traj)
        print(original_shape)
        traj=np.vstack(traj)
        print(np.shape(traj))
        dist=pairwise_distances(traj[::2])
        epsilon=np.min(dist[np.nonzero(dist)])
        print(epsilon)

        if derivative is False:
            k_epsilon=np.exp(-(traj[:,0]**2)/(epsilon))+np.exp(-(traj[:,1]**2)/(epsilon))+np.exp(-(traj[:,2]**2)/(epsilon))
            k_epsilon=np.reshape(k_epsilon)
            return k_epsilon
        else:
            grad_k_epsilon=np.exp(-(traj[:,0]**2)/(epsilon)) * (-2/epsilon*traj[:,0])+ np.exp(-(traj[:,1]**2)/(epsilon)) * (-2/epsilon*traj[:,1]) +np.exp(-(traj[:,2]**2)/(epsilon)) * (-2/epsilon*traj[:,2]) 
            grad_k_epsilon=np.reshape(grad_k_epsilon)
            return grad_k_epsilon
        
        
    def guess_dirac_delta(state_a,state_b, in_domain):
        r=np.zeros(shape= np.shape(in_domain))
        r[np.where(in_domain==0)[0]]=state_b [np.where(in_domain==0)[0]] + state_a[np.where(in_domain==0)[0]]
        return L 

    def compute_generator(x,y=None,dt=1,lag=1):
        if y is None :
            y=x
        N=np.shape(x)[0]
        K=1/N * x[:-lag] .dot(y[lag:])
        S=1/N * x .dot(x)
        L= K-S/(lag*dt)
        return L
    
    def compute_committor(basis, guess, grad_basis=None, force=None,coeff_q0=None):
        if coeff_q0 is None:
            fLf= compute_generator(basis)
            # h is 0
            frf=compute_generator(basis, guess)
            coeff_q0 = np.linalg.solve(fLf, frf)
            gamma = np.diag(coeff_q0) * basis
            g = gamma + guess
            return g
        else:
            fLf=compute_generator(basis)
            q0_diag=np.diag(coeff_q0)
            right_side=basis .dot(force) .dot(grad_basis).dot(q0_diag)
            gamma=np.linalg.solve(fLf, right_side)
            g=gamma + guess
            return g 
        

In [36]:
traj=np.load('../data/10000/traj_unperturb.npy')
state_a=np.load('../data/10000/stateA_unperturb.npy')
state_b=np.load('../data/10000/stateB_unperturb.npy')
in_domain=np.full(np.shape(state_b),1)-state_a - state_b

In [39]:
# step 1, build basis 
basis = build_gaussian_kernel(traj)
grad_basis=build_gaussian_kernel(traj,True)

(10000, 5, 2)
(50000, 2)


MemoryError: 